## Variation C : Insider Type Filter 

filter only CATEGORY = "Promoters" or contains "Promoter"

Question : Are promoter buys more reliable than employee buys ?

In [1]:
import pandas as pd 
import yfinance as yf 
from datetime import timedelta
from pandas import DatetimeIndex

In [2]:
# Load cleaned insider trading dataset
df = pd.read_csv("nse_insider_cleaned.csv")

# Clean column names
df.columns = df.columns.str.strip()

# only look for promoter trades 
df = df[df["category"].str.contains("Promoter", case=False, na=False)]

# Convert trade_date to datetime
df["trade_date"] = pd.to_datetime(df["trade_date"], errors="coerce")
df = df.dropna(subset=["trade_date"])

# Filter for large Buy trades
df = df[(df["type"] == "Buy") & (df["value"] > 10_00_000)]

# Prepare result list
results = []

print(f"🔍 Running backtest on {len(df)} trades...\n")

for i, row in df.iterrows():
    try:
        symbol = row["symbol"].strip() + ".NS"
        trade_date = row["trade_date"]
        entry_date = trade_date + timedelta(days=1)
        exit_date = trade_date + timedelta(days=30)

        # Download price data from yfinance
        ticker = yf.Ticker(symbol)
        data = ticker.history(start=entry_date - timedelta(days=5), end=exit_date + timedelta(days=5))

        # Skip if no data returned
        if data.empty:
            print(f"⚠️ No price data for {symbol}")
            continue

        # Ensure datetime index before removing timezone
        if isinstance(data.index, DatetimeIndex):
            try:
                data.index = data.index.tz_localize(None)
            except TypeError:
                pass
        else:
            print(f"⚠️ Skipping {symbol}: price data index not datetime")
            continue

        # Find the first valid trading day for entry and exit
        entry_row = data[data.index >= entry_date].head(1)
        exit_row = data[data.index >= exit_date].head(1)

        if entry_row.empty or exit_row.empty:
            continue

        entry_price = entry_row["Close"].values[0]
        exit_price = exit_row["Close"].values[0]
        return_pct = ((exit_price - entry_price) / entry_price) * 100

        results.append({
            "symbol": symbol,
            "trade_date": trade_date.date(),
            "entry_date": entry_row.index[0].date(),
            "exit_date": exit_row.index[0].date(),
            "entry_price": round(entry_price, 2),
            "exit_price": round(exit_price, 2),
            "return_%": round(return_pct, 2)
        })

    except Exception as e:
        print(f"⚠️ Error on row {i} ({row['symbol']}): {e}")
        continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv("variation_C_results.csv", index=False)
print("\n✅ Backtest complete. Results saved to 'variation_C_results.csv'.")

# Summary statistics
if not results_df.empty:
    avg_return = results_df["return_%"].mean()
    win_rate = (results_df["return_%"] > 0).mean() * 100
    print(f"\n📊 Strategy Summary:")
    print(f"Total Trades Analyzed: {len(results_df)}")
    print(f"Average Return: {avg_return:.2f}%")
    print(f"Win Rate       : {win_rate:.2f}%")
else:
    print("❌ No valid trades found for backtesting.")

🔍 Running backtest on 3747 trades...



$ADOR.NS: possibly delisted; no price data found  (1d 2024-12-19 00:00:00 -> 2025-01-27 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1734546600, endDate = 1737916200")
$ADOR.NS: possibly delisted; no price data found  (1d 2024-12-19 00:00:00 -> 2025-01-27 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1734546600, endDate = 1737916200")


⚠️ No price data for ADOR.NS
⚠️ No price data for ADOR.NS


$ARHAM.NS: possibly delisted; no timezone found


⚠️ No price data for ARHAM.NS


...

⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS


$SUPPETRO.NS: possibly delisted; no timezone found
$SUPPETRO.NS: possibly delisted; no timezone found


⚠️ No price data for SUPPETRO.NS
⚠️ No price data for SUPPETRO.NS


$UMIYA-MRO.NS: possibly delisted; no price data found  (1d 2022-02-10 00:00:00 -> 2022-03-21 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1644431400, endDate = 1647801000")


⚠️ No price data for UMIYA-MRO.NS

✅ Backtest complete. Results saved to 'variation_C_results.csv'.

📊 Strategy Summary:
Total Trades Analyzed: 3664
Average Return: 5.54%
Win Rate       : 62.55%
